In [31]:
import sys
import pathlib
import os
import psycopg2
import requests
import geopandas as gpd
import pandas as pd
import numpy as np
import database as db
import re
from xml.etree.ElementTree import ElementTree, Element, SubElement
import io
import json
import openpyxl
import csv
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles.alignment import Alignment
import apsim.wrapper as apsim
from apsim.daymet import Weather, create_excel_met, create_all_excel_met
from apsim.apsim_input_writer import get_date, add_management_year, create_mukey_runs
from apsim import run_apsim
from apsim.apsim_output_parser import parse_all_output, parse_summary_output
from budgets.budget_calcs import Budget
import munging
import traceback

In [2]:
dbconn = db.connect_to_db('database.ini')
raccoon_2018 = 'raccoon.raccoon_clu_ssurgo_2018'
#raccoon = pd.read_sql(f"SELECT * FROM {raccoon_2018} LIMIT 200;", dbconn)
#greene = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE County='Greene';", dbconn)
#greene_soils = pd.read_sql(f"SELECT DISTINCT mukey FROM {raccoon_2018} where County='Greene';", dbconn)
#field_25 = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE clukey=2515723;", dbconn)

In [43]:
spin_up_corn = json.loads( open( 'crop_jsons/maize.json', 'r' ).read() )
spin_up_soybean = json.loads( open( 'crop_jsons/soybean.json', 'r' ).read() )

### Step 1: Get the counties we are interested in.

In [6]:
#buena_vista = get_county(dbconn, 'raccoon.raccoon_clu_ssurgo_2018', 'IA021', "wkb_geometry")

### Step 2: Get the county centroid for creating APSIM met files

In [8]:
#bv_centroid = get_centroid(buena_vista, 'fips', "wkb_geometry")

In [10]:
# greene = get_county(dbconn, 'raccoon.raccoon_clu_ssurgo_2018', 'IA073', "wkb_geometry")
# greene_centroid = get_centroid(greene, 'fips', "wkb_geometry")
# create_met(greene_centroid[0], greene_centroid[1], 2012, 2019, 'greene')

### Step 3: Get all distinct counties and create met files for all county geometry lat/lon

In [15]:
#fips = get_distinct(dbconn, raccoon_2018, 'fips')
# create_all_met(dbconn, fips, raccoon_2018)
# create_all_excel_met(dbconn, fips, raccoon_2018)

### Step 4: Get clukey crop rotations

In [17]:
#rots = pd.read_sql('SELECT * FROM raccoon.raccoon_rots LIMIT 100;', dbconn)
#rots['rotation'] = ''
#group df by clukey, sort by year, then loop through and get rotation, appending to original df.
# clukeys = rots.drop_duplicates('clukey')
# for i in clukeys['clukey']:
#     field = rots.loc[rots['clukey'] == i]
#     rotation = get_rotation(field, 'crop')
# push to database
# db_table = 'raccoon_crop_rotations'
# db_schema = 'raccoon'
# rots.to_sql(
#         name = db_table,
#         con = dbconn,
#         schema = db_schema,
#         if_exists = 'replace',
#         index = False )

### Step 5: Get the soil properties for each individual mukey

In [18]:
#field_25 = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE clukey=2515723;", dbconn)
greene_clu = pd.read_sql(f"SELECT * FROM {raccoon_2018} WHERE County='Greene';", dbconn)
greene_soils = pd.read_sql(f"SELECT DISTINCT mukey FROM {raccoon_2018} where County='Greene';", dbconn)

### Loop through all rows - !!!DON'T RUN!!!

### Create APSIM files for the unique soils for a given county

In [ ]:
#greene_soils = pd.read_sql(f"SELECT DISTINCT mukey FROM {raccoon_2018} where County='Greene';", dbconn)
greene_soils_list = list(greene_soils['mukey'])

In [40]:
soils_test_list = (1453495, 1453495)

In [1]:
create_mukey_runs(soils_test_list, dbconn, 'sfc', 'Greene', 'IA073')
create_mukey_runs(soils_test_list, dbconn, 'cc', 'Greene', 'IA073')
create_mukey_runs(soils_test_list, dbconn, 'cfs', 'Greene', 'IA073')

NameError: name 'create_mukey_runs' is not defined

In [ ]:
# create_mukey_runs(greene_soils_list, dbconn, 'cfs', 'Greene', 'IA073')
# create_mukey_runs(greene_soils_list, dbconn, 'sfc', 'Greene', 'IA073')
# create_mukey_runs(greene_soils_list, dbconn, 'cc', 'Greene', 'IA073')

In [20]:
all_output = parse_all_output("C:\\Users\\mnowatz\\Documents\\Dev\\aepe\\analyses\\apsim_files\\Greene")

In [21]:
all_output.head()

,title,county,fips,mukey,rotation,date,day,year,soybean_yield,maize_yield,soybean_biomass,maize_biomass,corn_buac,soy_buac,fertiliser,surfaceom_c,subsurface_drain,subsurface_drain_no3,leach_no3
0,County_Greene_fips_IA073_mukey_1453495_rot_cc_sim,Greene,IA073,1453495,cc,2018-01-01,1,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1163.6864,NaN,NaN,0.0
1,County_Greene_fips_IA073_mukey_1453495_rot_cc_sim,Greene,IA073,1453495,cc,2018-02-01,2,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1163.6864,NaN,NaN,0.0
2,County_Greene_fips_IA073_mukey_1453495_rot_cc_sim,Greene,IA073,1453495,cc,2018-03-01,3,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1163.6864,NaN,NaN,0.0
3,County_Greene_fips_IA073_mukey_1453495_rot_cc_sim,Greene,IA073,1453495,cc,2018-04-01,4,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1163.6864,NaN,NaN,0.0
4,County_Greene_fips_IA073_mukey_1453495_rot_cc_sim,Greene,IA073,1453495,cc,2018-05-01,5,2018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1163.6864,NaN,NaN,0.0


In [22]:
summary_output = parse_summary_output("C:\\Users\\mnowatz\\Documents\\Dev\\aepe\\analyses\\apsim_files\\Greene")

In [23]:
summary_output.head()

,title,county,fips,mukey,rotation,year,soybean_yield,maize_yield,corn_buac,soy_buac,soybean_biomass,maize_biomass,fertiliser,surfaceom_c_init,surfaceom_c_end,leach_no3
0,County_Greene_fips_IA073_mukey_1453495_rot_cc_sim,Greene,IA073,1453495,cc,2018,0.00000,2107.11308,39.41541,0.00000,0.0000,9709.98004,220.0,1163.68640,1368.94153,69.11974
1,County_Greene_fips_IA073_mukey_1453495_rot_cfs...,Greene,IA073,1453495,cfs,2018,0.00000,2104.64886,39.36931,0.00000,0.0000,9710.55276,220.0,761.95184,1367.38245,58.45128
2,County_Greene_fips_IA073_mukey_1453495_rot_sfc...,Greene,IA073,1453495,sfc,2018,1831.33118,0.00000,0.00000,31.36418,4612.8584,0.00000,0.0,1149.34387,736.50665,98.54264
3,County_Greene_fips_IA073_mukey_1651354_rot_cc_sim,Greene,IA073,1651354,cc,2018,0.00000,7976.43613,149.20627,0.00000,0.0000,16842.03793,220.0,1428.02869,1682.92847,74.80633
4,County_Greene_fips_IA073_mukey_1651354_rot_cfs...,Greene,IA073,1651354,cfs,2018,0.00000,8021.76992,150.05429,0.00000,0.0000,17291.39834,220.0,958.68036,1758.85962,53.06663


In [27]:
db_schema = 'raccoon'
db_table = 'apsim_output_summary' 
summary_output.to_sql(
con = dbconn,
name = db_table,
schema = db_schema,
if_exists = 'replace',
index = False,
chunksize=1000,
method='multi' )

In [30]:
db_schema = 'raccoon'
db_table = 'apsim_output_all'
all_output.to_sql(
con = dbconn,
name = db_table,
schema = db_schema,
if_exists = 'replace',
index = False,
chunksize=1000,
method='multi' )

In [ ]:
#group by clukeys
#get clukey rotation
#look up yield for each mukey within clukey
#calculate profit
#append profit to clukey df

In [32]:
output_table = 'raccoon.apsim_output_summary'
county = 'Greene'
county_df = pd.read_sql(f"SELECT * FROM raccoon.raccoon_clu_ssurgo_2018 WHERE county = '{county}';", dbconn)
clukeys = list(county_df['clukey'].drop_duplicates())

In [ ]:
rot_df = pd.read_sql(f'SELECT * FROM raccoon.raccoon_rots WHERE clukey = 2516788;', dbconn).sort_values(by=['years'], ascending=True)

In [33]:
clukey_df = county_df.loc[county_df['clukey'] == 2516788]
clukey_df

,state,fips,huc8,clukey,musym,mukey,acres,clu_pct,wkb_geometry,county
982,IA,IA073,07100006,2516788,6,406246,0.05661,0.00169,0103000020E6100000010000000700000009C5492C538D...,Greene
983,IA,IA073,07100006,2516788,L507,2922024,0.01131,0.00034,0103000020E61000000100000004000000E5AD91649E8D...,Greene
35966,IA,IA073,07100006,2516788,L138B2,2877281,2.82280,0.08451,0103000020E6100000010000000F0000004043EC639E8D...,Greene
35967,IA,IA073,07100006,2516788,L138B,2765528,1.58281,0.04739,0103000020E6100000010000000D000000D6037B639E8D...,Greene
36037,IA,IA073,07100006,2516788,L107,2835017,3.08083,0.09223,0103000020E61000000100000029000000FDA681629E8D...,Greene
36038,IA,IA073,07100006,2516788,585B,406245,1.54089,0.04613,0103000020E61000000100000016000000A588B4639E8D...,Greene
36039,IA,IA073,07100006,2516788,585B,406245,3.98312,0.11925,0103000020E61000000100000025000000EA5B8850748D...,Greene
36040,IA,IA073,07100006,2516788,L138C2,2877326,2.22587,0.06664,0103000020E6100000010000001800000003C40EA4708D...,Greene
36041,IA,IA073,07100006,2516788,L138C2,2877326,1.77938,0.05327,0103000020E6100000010000001100000019A2D4639E8D...,Greene
36060,IA,IA073,07100006,2516788,L138B2,2877281,9.55006,0.28591,0103000020E61000000100000057000000995EE9377D8D...,Greene


In [ ]:
for i in clukeys[0:5]:
    try:
        rot_df = pd.read_sql(f'SELECT * FROM raccoon.raccoon_rots WHERE clukey = {i};', dbconn).sort_values(by=['years'], ascending=True)
        clukey_df = county_df.loc[county_df['clukey'] == i]
        rotation = get_rotation(rot_df, 'crop')
        for i in clukey_df.index():
            try:
                mukey_summary = pd.read_sql(f"SELECT * FROM raccoon.apsim_output_summary WHERE county = '{county}' AND mukey = {i} AND rotation = {rotation};")
                if rotation == 'cc' or rotation == 'cfs':
                    buac = mukey_summary['corn_buac']
                elif rotation == 'sfc':
                    buac = mukey_summary['soy_buac']
                else:
                    print('Not a corn or soybean rotation.')
                    continue
                acres = clukey_df.loc[i, 'acres']
                lbs_nitr = int(mukey_summary['fertiliser'])*0.892
                no3_loss = mukey_summary['leach_no3']


    except:
        print('Error occured')
        continue
    

In [ ]:
rot_df = pd.read_sql(f'SELECT * FROM raccoon.raccoon_rots WHERE clukey = 2518856;', dbconn).sort_values(by=['years'], ascending=True)

In [ ]:
county_df = pd.read_sql(f"SELECT * FROM raccoon.raccoon_clu_ssurgo_2018 WHERE county = 'Greene';", dbconn)

In [ ]:
clukey_df = county_df.loc[county_df['clukey'] == 2518856].drop_duplicates('mukey')

In [ ]:
county = 'Greene'